# Neural Networks II

Neurale Netze lassen sich sehr einach mittels 

- TensorFlow (https://www.tensorflow.org)
- Keras (https://keras.io)

erstellen, trainieren und anwenden. Keras ist dabei ein High-Level API (application programming interface), welches auf Tensorflow aufbaut. Diese Vorlesung führt daher zuerst die wichtigsten Schritte zur Erstellung
und zum Training von Neuronalen Netzen mittels Tensorflow ein. Im zweiten Schritt werden wir dann Keras näher beleuchten.

## Tensorflow

Die folgenden Abschnitte geben eine Step-by-Step Anleitung für die Erstellung und das Training Neuronaler Netze mittels Tensorflow. Wir beschränken uns hierbei auf das absolute Minimum, da wir später eine deutliche einfachere Vorgehensweise mit Keras kennenlernen werden. Folgende Informationen/Schritte sind dabei notwendig

(1) **Trainingsdaten**: $(\vec{x}^{(i)},y^{(i)})$ mit $i=1,\ldots,m$ und $\vec{x}^{(i)}\in\mathbb{R}^n$

(2) **Netzwerkarchitektur**: Die Schichten des Netzes haben die Größen $(n,h_1,h_2,\ldots,h_L,K)$, wobei die Größe der Eingabeschicht der Anzahl der features $n$ und die Größe der Ausgabeschicht der Anzahl der Klassen $K$ entspricht. Die $L$ Zwischenlayer haben die Größe $h_1,\ldots,h_K$. Zudem muss die Aktivierungsfunktion in jedem Layer spezifiziert werden.

(3) **Costfunction**: Mehrklassige Klassifikation mit der Crossentropy 
$$ 
J(\mathbf{W}_1,\ldots,\mathbf{W}_{K+1}) = -\sum_{i=1}^n\sum_{j=1}^K \mathbf{Y}_{ij}\log \mathbf{H}_{ij}
$$
oder Regression mit
$$ 
J(\mathbf{W}_1,\ldots,\mathbf{W}_{K+1}) = \frac{1}{m}\sum_{i=1}^n(y^{(i)}_{\mathrm{pred}}-y^{(i)})^2
$$

(4) **Optimierverfahren**: Wir werden die learning rate $\alpha$ und das Optimierverfahren (wir nutzen bei Keras das RMSprop Verfahren - siehe Vorlesung) festgelegt.

(5) **Training**/**Evaluaring**:
Für das Training der Netzes verwenden wir jeweils das Mini-Batch gradient descent Verfahren - siehe Vorlesung.

In [1]:
import numpy as np
import tensorflow as tf

In [ ]:
tf.__version__

### (1) Trainingsdaten

Wir lesen als Trainingsdaten das MNIST Datenset ein und splitten die 70.000 Bilder in

- 55.000: Training
- 10.000: Test
- 5.000: Validation

Das bisher noch nicht angesprochene Validation Set wird beim Machine Learning zur Überprüfung der Hypterparameter (in unserem Falle sind dies
die Anzahl der Hiddenlayer $L$ und deren Größen $h_1,\ldots,h_L$).

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# 70.000 Bilder 
# - 55.000 Training
# - 10.000 Testing
# - 5000 Validation

X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)

X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_test = y_test.astype(np.int32)

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

### (2) Netzwerkarchitektur

Im folgenden legen wir die Netzwerkarchitektur fest. Wir nutzen $L=1$
mit $h_1=100$ (zudem ist bei MNIST $n=28^2=784$ und $K=10$).

In [ ]:
n = 28*28  # MNIST
h_1 = 100
K = 10

In [ ]:
X = tf.placeholder(tf.float32, shape=(None, n))
y = tf.placeholder(tf.int32, shape=(None))

In [ ]:
hidden1 = tf.layers.dense(X,h_1, activation=tf.nn.relu)
logits = tf.layers.dense(hidden1, K)

### (3) Costfunction

In [ ]:
cost_function = tf.losses.sparse_softmax_cross_entropy(y,logits)

### (4) Optimierverfahren

In [ ]:
learning_rate = 0.01

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_op = optimizer.minimize(cost_function)

### (5) Training/Evaluatierung

#### (a) Berechnug der accuracy

In [ ]:
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

#### (b) Initalisiering der Variablen

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

#### (c) Festlegung der Iterationschritte und des batch_sizes

In [ ]:
iterations = 10
batch_size = 32

In [ ]:
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

#### (d)  Tensorflow session eröffnen und Training durchführen

In [ ]:
#with tf.Session(config=config) as sess:
with tf.Session() as sess:
    init.run()
    for n in range(iterations):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})           
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print("Iteration:", n, "Batch accuracy:", acc_batch, "Val accuracy:", acc_val)
    save_path = saver.save(sess, "./my_model_final.ckpt")

####  (e) Accuracy bezüglich Test Set berechnen

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt") 
    Z = logits.eval(feed_dict={X: X_test})
    y_pred = np.argmax(Z, axis=1)
print(np.mean(y_pred==y_test))

## Keras

Mithilfe der High-Level API Keras lässt sich das obere Prozedere sehr einfach, elegant und nutzerfreundlich implementieren. Dazu importieren wir Keras und die wichtigsten Funktionen zur Erstellung des obigen Neuronalen Netzes.

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


Wir führen nun die oben genannten 5 Schritte durch:
#### (1) Trainingsdaten: 

Die Trainingsdaten bleiben unverändert.

X.shape = (m,n)

y.shape = (m,)

#### (2) Netzwerkarchitektur:

Das Model lässt sich hier sehr einfach mittels des Sequential Frameworks erstellen (https://keras.io/getting-started/sequential-model-guide/).

Die zur Verfügung stehenden Aktivierungsfunktionen finden sich hier: https://keras.io/api/layers/activations/ .

In [4]:
n = 28*28
h1 = 100
h2 = 100
h3 = 100
h4 = 100
K = 10

model = Sequential()
model.add(Dense(h1, activation='relu', input_dim=n))
model.add(Dense(h2, activation='relu'))
model.add(Dense(h3, activation='relu'))
model.add(Dense(h4, activation='relu'))
model.add(Dense(K, activation='softmax'))

model.summary()

W0604 11:51:43.640750 139947246712640 module_wrapper.py:139] From /opt/tljh/user/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0604 11:51:43.649227 139947246712640 module_wrapper.py:139] From /opt/tljh/user/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0604 11:51:43.658878 139947246712640 module_wrapper.py:139] From /opt/tljh/user/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 109,810
Trainable params: 109,810
Non-trainable params: 0
_________________________________________________________________


#### (3) und (4) Costfunction und Optimierverfahren

Wir wählen hier beispielhaft 

- Optimierer:  RMSprop aus https://keras.io/optimizers .  
- Costfunction: Wir verwenden sparse_categorical_crossentropy aus https://keras.io/api/losses , welche Integer-Werte als Targets verarbeiten kann (die categorical_crossentropy arbeitet mit Targets in der One-Hot-Encoding Darstellung)
- Metrik: accuracy aus https://keras.io/metrics .

In [9]:
model.compile(optimizer=keras.optimizers.adam(),
              loss=keras.losses.sparse_categorical_crossentropy,
              metrics=['sparse_categorical_accuracy'])

In [11]:
X_train.shape[0]/32

1718.75

#### (5) Training/Evaluierung

Für das Trainig verwenden wir fit Routine (https://keras.io/models/sequential/#fit), welcher wir die 
Trainingsdaten, die Anzahl der Epochen, den Mini-Batch-Size, sowie das Validierungs-Set übergeben.

In [12]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_valid,y_valid))

W0604 12:18:21.389195 139947246712640 deprecation.py:323] From /opt/tljh/user/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0604 12:18:21.421484 139947246712640 module_wrapper.py:139] From /opt/tljh/user/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0604 12:18:21.529806 139947246712640 module_wrapper.py:139] From /opt/tljh/user/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0604 12:18:21.639579 139947246712640 module_wrapper.py:139] From /opt/tljh/user/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use

Train on 55000 samples, validate on 5000 samples
Epoch 1/20
55000/55000 [==============================] - 8s 144us/step - loss: 0.2679 - sparse_categorical_accuracy: 0.9183 - val_loss: 0.1495 - val_sparse_categorical_accuracy: 0.9554
Epoch 2/20
55000/55000 [==============================] - 7s 135us/step - loss: 0.1215 - sparse_categorical_accuracy: 0.9625 - val_loss: 0.1026 - val_sparse_categorical_accuracy: 0.9698
Epoch 3/20
55000/55000 [==============================] - 7s 136us/step - loss: 0.0905 - sparse_categorical_accuracy: 0.9717 - val_loss: 0.1163 - val_sparse_categorical_accuracy: 0.9656
Epoch 4/20
55000/55000 [==============================] - 7s 135us/step - loss: 0.0714 - sparse_categorical_accuracy: 0.9780 - val_loss: 0.1023 - val_sparse_categorical_accuracy: 0.9732
Epoch 5/20
55000/55000 [==============================] - 7s 134us/step - loss: 0.0605 - sparse_categorical_accuracy: 0.9812 - val_loss: 0.0916 - val_sparse_categorical_accuracy: 0.9732
Epoch 6/20
55000/5500

- Zur Evaluierung bzg. des Testsets verwenden wir die Funktion evaluate (https://keras.io/models/model/#evaluate) ...

In [ ]:
model.evaluate(X_test,y_test)

- ... oder predict_classes 

In [ ]:
y_test_p = model.predict_classes(X_test)
np.mean(y_test_p==y_test)

- Die Accuracy von Training- und Validierungset lässt sich wie folgt plotten.

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())

In [ ]:
plt.plot(history.history['sparse_categorical_accuracy'],'o')
plt.plot(history.history['val_sparse_categorical_accuracy'],'o')

In [ ]:
plt.plot(history.history['loss'],'o')
plt.plot(history.history['val_loss'],'o')